In [1]:
import {InferenceSession, Tensor, env} from 'onnxruntime-web';
const ndarray = require('ndarray')
const ops = require('ndarray-ops')
const fs = require('fs')
import Jimp from "jimp";
import { Niivue, NVImage } from "@niivue/niivue";

In [2]:
// in kernel, The configuration runs using node, and not in the browser,
// which is why window is not defined
// we need to define it to make the code work

// global.window = { navigator: { userAgent: 'node.js' } };
// typeof(window) === 'undefined'
const volume = await NVImage.loadFromUrl({url:"https://niivue.github.io/niivue-demo-images/mni152.nii.gz"})
console.log(volume)

NVImage {
  DT_NONE: 0,
  DT_UNKNOWN: 0,
  DT_BINARY: 1,
  DT_UNSIGNED_CHAR: 2,
  DT_SIGNED_SHORT: 4,
  DT_SIGNED_INT: 8,
  DT_FLOAT: 16,
  DT_COMPLEX: 32,
  DT_DOUBLE: 64,
  DT_RGB: 128,
  DT_ALL: 255,
  DT_INT8: 256,
  DT_UINT16: 512,
  DT_UINT32: 768,
  DT_INT64: 1024,
  DT_UINT64: 1280,
  DT_FLOAT128: 1536,
  DT_COMPLEX128: 1792,
  DT_COMPLEX256: 2048,
  DT_RGBA32: 2304,
  name: 'mni152.nii.gz',
  id: 'f40c00ec-7ede-44b5-8b4d-2bdaf8bd6619',
  _colormap: 'gray',
  _opacity: 1,
  percentileFrac: 0.02,
  ignoreZeroVoxels: false,
  trustCalMinMax: true,
  colormapNegative: '',
  colormapLabel: [],
  frame4D: 0,
  cal_minNeg: NaN,
  cal_maxNeg: NaN,
  colorbarVisible: true,
  visible: true,
  modulationImage: null,
  modulateAlpha: 0,
  series: [],
  onColormapChange: [Function (anonymous)],
  onOpacityChange: [Function (anonymous)],
  hdr: si {
    littleEndian: true,
    dim_info: 0,
    dims: [
      3, 207, 256, 215,
      1,   1,   1,   1
    ],
    intent_p1: 0,
    intent_p2: 0,


In [3]:
function normalizeArray(
    array: Float32Array,
    max: number
  ): Float32Array {
    let normalizedArray = new Float32Array(array.length);
    for (let i = 0; i < array.length; i++) {
      normalizedArray[i] = array[i] / max;
    }
    return normalizedArray;
  }
  
  function stackSliceToRGB(
    buffer: Float32Array | Uint8Array,
  ): Float32Array {
    let bufferLength = buffer.length,
      result = new Float32Array(bufferLength * 3);
  
    for (let i = 0; i < bufferLength; i++) {
      result[3 * i] = buffer[i];
      result[3 * i + 1] = buffer[i];
      result[3 * i + 2] = buffer[i];
    }
    return result;
  }
  

In [4]:
const preprocessVolume = (image) => {
    try {
      const imageRAS = image.img2RAS();
      console.log("image.calMax, image.calMin ", image.global_max, image.global_min);

      const normalizedArray = normalizeArray(imageRAS, image.global_max);
      console.log("normalizedArray ", normalizedArray);
      return normalizedArray;
    } catch (e) {
      console.log(`failed to preprocess volume: ${e}. `);
      throw Error(`failed to preprocess volume: ${e}. `);
    }
  }

  const preprocess = async (slice2D: Float32Array, sliceId: number, width: number, height: number): Promise<Float32Array> => {
    try {
      console.log("imageArray ", slice2D);
      let image3Channels: Float32Array;
      const sliceArray = slice2D.slice(
        width * height * sliceId,
        width * height * (sliceId + 1),
      );
      console.log("sliceArray ", sliceArray.reduce((a, b) => a + b, 0));
  
      image3Channels = stackSliceToRGB(sliceArray);
      console.log("image3Channels ", image3Channels, image3Channels.reduce((a, b) => a + b, 0));
      return image3Channels;
    } catch (e) {
      console.log(`failed to inference ONNX model: ${e}. `);
      throw Error(`failed to inference ONNX model: ${e}. `);
    }
  };


**Creating a 3D array of size (2, 9, 5)**
```python
array_3d = np.array([
    [
        [1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [41, 42, 43, 44, 45]
    ],
    [
        [46, 47, 48, 49, 50],
        [81, 82, 83, 84, 85],
        [86, 87, 88, 89, 90]
    ]
])
```

**Get flatten array from index [:,:,1]**
```python
array_3d[:,:,1].flatten() 
```

[ 2  7 42 47 82 87]

=**Get flatten array from index [:,1,:]**
```python
array_3d[:,1,:].flatten()
```

[ 6  7  8  9 10 81 82 83 84 85]

**Get flatten array from index [1,:,:]**
```python
array_3d[1,:,:].flatten()
```

[46 47 48 49 50 81 82 83 84 85 86 87 88 89 90]


In [5]:
// function plotArray(array, dims) {
//     // 1. convert array to Uint8 or Float32
//     let arrayToPlot = new Uint8Array(array);
//     // 2. save to png
//     imagedataToImage(arrayToPlot, dims[0], dims[1]);
// }

In [6]:
function nvVolumetoSlice(volume, dims, fov, index): any {
    let sliceArray;
    switch (fov) {
        // sagittal dimsRAS[1], coronal dimsRAS[2], axial dimsRAS[3]
        
        // 1. if axial, slice the array from volume
        case "axial":
            sliceArray = volume.slice(
                dims[0] * dims[1] * index,
                dims[0] * dims[1] * (index + 1),
            );
            console.log("axial sliceArray",dims[1], dims[2], index, sliceArray.length);
        // 2. if coronal, 
        case "coronal":
        // 3. if sagittal,
        case "sagittal":
    }
    return sliceArray;
    
}

function nvSliceToTensor(nvSlice, dims): any {
    console.log("nvSlice", nvSlice.length);
    // 1. preprocess
    let image3Channels = stackSliceToRGB(nvSlice);
    // 2. convert to float32
    console.log("image3Channels", image3Channels.length);
    // 3. tensor
    const inputTensor = new Tensor("float32", image3Channels, dims);
    return inputTensor;
}

In [7]:
const dims = [volume.dimsRAS[1], volume.dimsRAS[2], volume.dimsRAS[3]];
let nvVolume = preprocessVolume(volume);
let nvSlice = nvVolumetoSlice(nvVolume, dims, "axial", 90);
let inputTensor = nvSliceToTensor(nvSlice, [1, 3, dims[0], dims[1]]);

image.calMax, image.calMin  80 40
normalizedArray  Float32Array(11393280) [
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0,
  ... 11393180 more items
]
axial sliceArray 256 215 90 52992
nvSlice 52992
image3Channels 158976


In [29]:
console.log(inputTensor.ImgData, dims[0], dims[1])
console.log(dst.cols, dst.rows, dst.data);


undefined 207 256
256 207 Uint8Array(635904) [
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0,
  ... 635804 more items
]


In [27]:
import { cv } from 'opencv-wasm'

// await cv.loadOpenCV();
let dst = cv.matFromArray(dims[0], dims[1], cv.CV_32FC3, inputTensor.data);
console.log(dst);
// new Jimp({
//     width: dst.cols,
//     height: dst.rows,
//     data: Buffer.from(dst.data)
// }).write('/home/thuy/repo/deepsyence/dilation.png');

Mat {}


In [9]:
console.log("inputTensor", inputTensor.data.length, inputTensor.dims);

inputTensor 158976 [ 1, 3, 207, 256 ]


In [10]:
global.self = global;
env.wasm.numThreads = 1;

const encoderSession = await InferenceSession.create('/home/thuy/repo/deepsyence/public/model/efficient_sam_vitt_encoder.onnx');
const decoderSession = await InferenceSession.create('/home/thuy/repo/deepsyence/public/model/efficient_sam_vitt_decoder.onnx');


In [11]:
async function runModel(model, preprocessedData): Promise<[Tensor, number]> {
    const start = new Date();
    try {
      const feeds: Record<string, Tensor> = {};
      feeds[model.inputNames[0]] = preprocessedData;
      const outputData = await model.run(feeds);
      const end = new Date();
      const inferenceTime = (end.getTime() - start.getTime());
      const output = outputData[model.outputNames[0]];
      return [output, inferenceTime];
    } catch (e) {
      console.error(e);
      throw new Error();
    }
  }

In [12]:
const [res, time] =  await runModel(encoderSession, inputTensor);
var output = res.data;
var inferenceTime = time;
// var results = softmax(Array.prototype.slice.call(output))

In [13]:
console.log((output as Float32Array).reduce((a,b) => a + b, 0), inferenceTime);

1831.3217101244136 6015
